In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, confusion_matrix
from PIL import Image
import os
import net
from face_alignment import align
from IPython.display import display, clear_output

In [ ]:
def load_pretrained_model(architecture='ir_101'):
    # load model and pretrained statedict
    assert architecture in model_name.keys()
    model = net.build_model(architecture)
    statedict = torch.load(model_name[architecture])['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model

In [ ]:
def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img[:,:,::-1] / 255.) - 0.5) / 0.5
    tensor = torch.tensor([brg_img.transpose(2,0,1)]).float()
    return tensor

In [ ]:
model_name = {
    'ir_101':"pretrained/adaface_ir101_ms1mv2.ckpt",
}

model = load_pretrained_model()

In [ ]:
X = []
y = []
with open('./benchmark/dataset/cplfw/pairs_CPLFW.txt','r') as f:
    lines = f.readlines()
    for i in range(0, len(lines), 2):
        img1, label1 = lines[i].strip().split()
        img2, label2 = lines[i+1].strip().split()
        img1 = "./benchmark/dataset/cplfw/aligned images/"+img1
        img2 = "./benchmark/dataset/cplfw/aligned images/"+img2
        X.append((img1, img2))
        y.append(label1 == '1')

X = np.array(X)
y = np.array(y, dtype=bool)

In [ ]:
X.shape, y.shape

In [ ]:
def distance(img1, img2, model, aligned=True):
    if aligned:
        aligned_face_img1 = Image.open(img1).resize((112, 112))
        aligned_face_img2 = Image.open(img2).resize((112, 112))
    else:
        aligned_face_img1 = align.get_aligned_face(img1)
        aligned_face_img2 = align.get_aligned_face(img2)
    
    if aligned_face_img1 is None or aligned_face_img2 is None:
        print("Aligned face not found in one or both images.")
        return None

    # Prepare input tensors
    tensor_img1 = to_input(aligned_face_img1)
    tensor_img2 = to_input(aligned_face_img2)
    
    # Get features
    feature_img1, _ = model(tensor_img1)
    feature_img2, _ = model(tensor_img2)
    
    # Compute similarity
    cos_sim = torch.nn.functional.cosine_similarity(feature_img1, feature_img2)
    similarity_score = cos_sim.item()
    
    return similarity_score

In [ ]:
distance(X[11,0],X[11,1], model)
distance(X[5998,0],X[5998,1], model, aligned=False)

In [ ]:
threshold = 0.17

y_hat = []

for i in range(len(X)):
    clear_output(wait=True)
    display(i)
    dist = distance(X[i,0],X[i,1], model)
    if dist is None or dist < threshold:
        y_hat.append(False)
    else:
        y_hat.append(True)

y_hat = np.array(y_hat)

accuracy_score(y, y_hat)